# Main Pipleline

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

Cameras needed to be calibrated first to remove distortion

In [ ]:
import calibrate_camera # calibration module

In [ ]:
mtx, dist = calibrate_camera.calibrate(9, 6, 'camera_cal/*.jpg')

after we got the matrix and distortion coefficients we apply them to checkboards images to check the results

In [ ]:
import glob
import cv2
images_names = glob.glob('camera_cal/*.jpg')
for index, image_name in enumerate(images_names):
    image = cv2.imread(image_name)
    image_size = (image.shape[1], image.shape[0])
    calibrate_camera.undistort(image, mtx, dist, 
                               'output_images/calibrated_boards/test{}.jpg'.format(index))

we save the results in a pickle dictionary so we don't need to repeat the pipeline and gain performance improvements

In [ ]:
import pickle
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/wide_dist_pickle.p", "wb" ) )

In [ ]:
images_names = glob.glob('test_images/*.jpg')
for index, image_name in enumerate(images_names):
    image = cv2.imread(image_name)
    image_size = (image.shape[1], image.shape[0])
    calibrate_camera.undistort(image, mtx, dist, 
                               'output_images/calibrated_roads/road{}.jpg'.format(index))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import sobel # sobel module reads images in RGB using matplot lib
images_names = glob.glob('output_images/calibrated_roads/*.jpg')
ksize = 3
for index, image_name in enumerate(images_names):
    image = mpimg.imread(image_name)
    combined_binary = sobel.get_binary(image, ksize)
    plt.imsave('output_images/roads_binary/{}.jpg'.format(index), combined_binary,
               cmap='gray')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import bird_view
import lanes
import cv2

images_names = glob.glob('output_images/roads_binary/*.jpg')
undists = glob.glob('output_images/calibrated_roads/*.jpg')
for index, image_name in enumerate(images_names):
    image = mpimg.imread(image_name)
    binary_warped, matrix, matrix_inv = bird_view.get_bird_view(image)
    plt.imsave('output_images/roads_view/{}.jpg'.format(index), binary_warped,
               cmap='gray')
    
    out_img, left_fitx, right_fitx, ploty = lanes.fit_polynomial(binary_warped)
    undist = plt.imread(undists[index])
    result = lanes.draw_path(binary_warped, left_fitx, right_fitx, ploty, matrix_inv, undist)
    
    plt.imsave('output_images/roads_maped/{}.jpg'.format(index), result)